# Inference from Scratch

All in one notebook.

In [1]:
from mlops2_with_dagster.utils import get_project_dir, printse
from pathlib import Path
project = 'mlops2_with_dagster'
project_dir = get_project_dir(project)
printse(f'project_dir: {project_dir}')

>>>> project_dir: /Users/rahul/Websites/mlops2_with_dagster


In [2]:
import pandas as pd
from joblib import dump, load
from hamilton import driver, base

In [3]:
from mlops2_with_dagster import features_pipeline, encoder_pipeline, model_pipeline

In [4]:
index_col = 'passengerid'
target_col = "survived"
config_model = {
    'index_column': index_col,
    'target_column': target_col,
    'random_state': 42,
    'max_depth': None,
    'validation_size_fraction': 0.33,
    't': 0.5
}

## Inference 2: Inference from the very beginning of the data

In [5]:
encoder_file: Path = project_dir/"intermediate_data/encoder.joblib"
data: Path = project_dir/"data/test.csv"
datatype: str = "dataset"
model_file: Path = project_dir/"models/rf.joblib"
infer_type: str = "dataset"

In [6]:
# parameters
df = pd.read_csv(data)
encoders = load(encoder_file)
datatype: str = "train"
infer_type: str = "dataset"
fit_clf = load(model_file)['fit_clf']

In [7]:
config_infer2 = {
    'index_column': index_col,
    'target_column': target_col,
    't': 0.5
}


In [10]:
infer2_adapter = base.SimplePythonGraphAdapter(base.DictResult())
infer2_dr = driver.Driver(config_infer2, 
                            features_pipeline, model_pipeline, encoder_pipeline,
                            adapter = infer2_adapter)
dinfer2 = dict(
    df = df,
    clf = fit_clf,
    **encoders['encoders']
)

infer2_dr.visualize_execution(['chain_predictions'],
                                       f"{project_dir}/artifacts/infer2_{infer_type}_scratch.dot", 
                                       {}, 
                                       inputs = dinfer2
)

In [11]:
infer2dict = infer2_dr.execute(['chain_predictions'],inputs = dinfer2)

<class 'pandas.core.frame.DataFrame'>


In [12]:
infer2dict

{'chain_predictions': (array([0.  , 0.72, 0.93, ..., 0.04, 0.98, 0.81]),
  array([0, 1, 1, ..., 0, 1, 1]))}

In [13]:
dump(infer2dict, f"{project_dir}/results/infer2_{infer_type}.joblib")

['/Users/rahul/Websites/mlops2_with_dagster/results/infer2_dataset.joblib']

In [14]:
import dagstermill

dagstermill.yield_result(infer2dict, output_name="inference_results")

{'chain_predictions': (array([0.  , 0.72, 0.93, ..., 0.04, 0.98, 0.81]),
  array([0, 1, 1, ..., 0, 1, 1]))}